# TranSysTor IDE Avancé 🔷

**Framework TSCP v0.2.0**

Environnement de Développement Intégré complet avec :
- 🎲 Cubes imbriqués CM1/CM2/CM3
- 📐 Plan CM0 (5×5)
- 🌐 Multilingue FR/EN
- 📊 Export OWL/RDFS/SHACL
- 💬 Chatbot IA
- 🧮 Validation mathématique

## 1. Imports des modules

In [1]:
# Imports des modules TranSysTor
from transystor_core import *
from transystor_viz import *
from transystor_export import *
from transystor_chatbot import *

# Imports standard
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Initialiser l'état
state = IDEState()

print("✅ Tous les modules chargés")
print(f"📍 Langue: {state.language}")
print(f"📍 Couche par défaut: {state.exclusive_layer}")

ModuleNotFoundError: No module named 'transystor_core'

## 2. Données - Principes rangés

In [ ]:
# Charger ou définir les principes
principles_data = [
    # CM0
    {'name': 'Processus', 'layer': 'CM0', 'position': [1, 1, -0.5], 'color': '#ef4444',
     'type': 'MetaMetaClass ∈ CM0', 'description': 'Transformation dans le temps'},
    {'name': 'Structure', 'layer': 'CM0', 'position': [2, 2, -0.5], 'color': '#f59e0b',
     'type': 'MetaMetaClass ∈ CM0', 'description': 'Organisation spatiale ou conceptuelle'},
    {'name': 'Échange', 'layer': 'CM0', 'position': [3, 3, -0.5], 'color': '#10b981',
     'type': 'MetaMetaClass ∈ CM0', 'description': 'Transfert d\'information, matière ou énergie'},
    
    # CM1
    {'name': 'Observateur', 'layer': 'CM1', 'position': [1.5, 1.5, 1.5], 'color': '#3b82f6',
     'type': 'MetaClass ∈ CM1', 'description': 'Point de départ anthropocentrique'},
    {'name': 'Interface', 'layer': 'CM1', 'position': [2, 1.5, 2], 'color': '#10b981',
     'type': 'MetaClass ∈ CM1', 'description': 'Médiation entre systèmes'},
    {'name': 'Réseau', 'layer': 'CM1', 'position': [2.5, 2, 2.5], 'color': '#ef4444',
     'type': 'MetaClass ∈ CM1', 'description': 'Structure organisationnelle'},
    
    # CM2
    {'name': 'Protocole', 'layer': 'CM2', 'position': [2, 1, 1], 'color': '#a855f7',
     'type': 'Class ∈ CM2', 'description': 'Instance d\'Interface avec règles', 'derives': ['Interface ⊂ CM1']},
    {'name': 'Bus', 'layer': 'CM2', 'position': [1, 3, 3], 'color': '#f97316',
     'type': 'Class ∈ CM2', 'combination': 'Bus = Processus ⊗ Distribution',
     'description': 'Transport cyclique', 'derives': ['Processus ⊂ CM0', 'Distribution ⊂ CM1']},
]

print(f"📦 {len(principles_data)} principes chargés")
for layer in ['CM0', 'CM1', 'CM2', 'CM3']:
    count = len([p for p in principles_data if p['layer'] == layer])
    print(f"   {layer}: {count} éléments")

## 3. Visualisation 3D Interactive

In [ ]:
# Contrôles de visualisation
layer_checkboxes = {
    'CM0': widgets.Checkbox(value=False, description='CM0 (Plan 5×5)'),
    'CM1': widgets.Checkbox(value=True, description='CM1 (Cube 3×3×3)'),
    'CM2': widgets.Checkbox(value=True, description='CM2 (Cube 4×4×4)'),
    'CM3': widgets.Checkbox(value=False, description='CM3 (Cube 5×5×5)')
}

exclusive_mode = widgets.Dropdown(
    options=[('Mode superposé', None), ('CM0 uniquement', 'CM0'),
             ('CM1 uniquement', 'CM1'), ('CM2 uniquement', 'CM2'),
             ('CM3 uniquement', 'CM3')],
    value='CM2',
    description='Affichage:'
)

language_selector = widgets.ToggleButtons(
    options=[('🇫🇷 Français', 'fr'), ('🇬🇧 English', 'en')],
    value='fr',
    description='Langue:'
)

show_grid_checkbox = widgets.Checkbox(value=True, description='Grille 3D')
show_axes_checkbox = widgets.Checkbox(value=True, description='Axes IJK')

update_button = widgets.Button(
    description='🔄 Actualiser',
    button_style='primary',
    icon='refresh'
)

viz_output = widgets.Output()

def update_visualization(b=None):
    state.language = language_selector.value
    state.exclusive_layer = exclusive_mode.value
    state.show_grid = show_grid_checkbox.value
    state.show_axes = show_axes_checkbox.value
    
    for layer, checkbox in layer_checkboxes.items():
        state.visible_layers[layer] = checkbox.value
    
    with viz_output:
        clear_output(wait=True)
        fig = create_nested_cubes_visualization(
            principles_data,
            state.visible_layers,
            state.exclusive_layer,
            state.show_grid,
            state.show_axes,
            state
        )
        fig.show()

update_button.on_click(update_visualization)

controls = widgets.VBox([
    widgets.HTML("<h3 style='color:white;'>⚙️ Contrôles</h3>"),
    language_selector,
    exclusive_mode,
    widgets.HTML("<h4 style='color:white;'>Couches visibles:</h4>"),
    *layer_checkboxes.values(),
    widgets.HTML("<h4 style='color:white;'>Options:</h4>"),
    show_grid_checkbox,
    show_axes_checkbox,
    update_button,
    viz_output
])

display(controls)
update_visualization()

## 4. Export Sémantique

In [ ]:
export_format = widgets.Dropdown(
    options=['OWL (Turtle)', 'SHACL', 'RDFS'],
    description='Format:'
)

export_button = widgets.Button(
    description='📊 Générer export',
    button_style='success',
    icon='download'
)

export_output = widgets.Output()

def on_export_clicked(b):
    with export_output:
        clear_output()
        
        if export_format.value == 'OWL (Turtle)':
            content = export_to_owl(principles_data)
            filepath = save_export(content, 'owl')
        elif export_format.value == 'SHACL':
            content = export_to_shacl(principles_data)
            filepath = save_export(content, 'shacl')
        else:
            content = export_to_rdfs(principles_data)
            filepath = save_export(content, 'rdfs')
        
        print(f"\n📄 Aperçu (500 premiers caractères):")
        print("=" * 60)
        print(content[:500] + "...")

export_button.on_click(on_export_clicked)

display(widgets.VBox([
    widgets.HTML("<h3 style='color:white;'>📊 Export Sémantique</h3>"),
    export_format,
    export_button,
    export_output
]))

## 5. Assistant IA (Chatbot)

In [ ]:
# Créer l'interface du chatbot
provider, api_key, chat_input, chat_button, chat_output = create_chatbot_interface(state)

# Questions prédéfinies
questions = get_predefined_questions()
questions_html = "<ul style='color:lightgray;'>"
for label, q in questions:
    questions_html += f"<li><b>{label}:</b> {q}</li>"
questions_html += "</ul>"

chatbot_interface = widgets.VBox([
    widgets.HTML("<h3 style='color:white;'>💬 Assistant IA</h3>"),
    widgets.HTML("<p style='color:gray;'>Validation, critique et suggestions</p>"),
    provider,
    api_key,
    widgets.HTML("<h4 style='color:white;'>Questions prédéfinies:</h4>"),
    widgets.HTML(questions_html),
    chat_input,
    chat_button,
    chat_output
])

display(chatbot_interface)

## 6. Validation Mathématique

In [ ]:
# Calculer l'orthogonalité
ortho_score, ortho_matrix = compute_orthogonality(principles_data)

print(f"📊 Score d'orthogonalité global: {ortho_score:.3f}")
print(f"   (1.0 = parfait, 0.0 = colinéaire)")
print(f"\n{'✅ PASS' if ortho_score > 0.6 else '⚠️  À REVOIR'}")

if ortho_score < 0.6:
    print("\n⚠️  Principes potentiellement trop proches.")
    print("   Suggestion: Vérifier les positions ou restructurer.")

## 7. Sauvegarde

In [ ]:
save_button = widgets.Button(
    description='💾 Sauvegarder l\'état complet',
    button_style='success'
)

save_output = widgets.Output()

def on_save_clicked(b):
    with save_output:
        clear_output()
        filepath = save_complete_state(state, principles_data)
        print(f"\n📁 Fichier: {filepath}")

save_button.on_click(on_save_clicked)

display(widgets.VBox([save_button, save_output]))

---

## ✅ Fonctionnalités complètes

- 🎲 Cubes imbriqués CM1⊂CM2⊂CM3
- 📐 Plan CM0 (5×5)
- 🌐 Multilingue FR/EN
- 📊 Export OWL/RDFS/SHACL
- 💬 Chatbot configurable
- 🧮 Validation mathématique
- 💾 Sauvegarde/chargement

---

*TranSysTor IDE v0.2.0 - Framework TSCP*